In [2]:
! pip install numpy==1.18.5

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 9.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.2 requires numpy>=1.20, but you have numpy 1.18.5 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.18.5 which is incompatible.
pandas 2.0.3 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.18.5 which is incompatible.
scikit-image 0.21.0 requires numpy>=1.21.1, but you have numpy 1.18.5 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 1.18.5 which is incompatible.
tensorflow 2.4.1 requires absl-py~=0.10, but 

# 0. aug test

In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os

image_data_gen =  ImageDataGenerator(
    rotation_range=150,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range = 0.1,
    shear_range=0,
    horizontal_flip=True
)

p = 'cell_data/1703121298-0001-G_R___10___red_blood_cell.png'
img = load_img( p )

img_array = img_to_array(img)
img_array = img_array.reshape( (1, img_array.shape[0], img_array.shape[1], img_array.shape[2]) )

i = 0
prefix='aug__'+os.path.basename(p).replace('.png','')
for batch in image_data_gen.flow(img_array, batch_size=1,save_to_dir='test_aug', save_prefix=prefix, save_format='png'):
    i = i+1
    if i>=50:
        break


2023-09-10 09:29:02.732077: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/root/anaconda3/envs/env_yolo_test01_py38/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.18.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os


def augment_one_image(img_file_path, dst_dir, num):
    image_data_gen =  ImageDataGenerator(
        rotation_range=150,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range = 0.1,
        shear_range=0,
        horizontal_flip=True
    )

    p = img_file_path
    img = load_img( p )

    img_array = img_to_array(img)
    img_array = img_array.reshape( (1, img_array.shape[0], img_array.shape[1], img_array.shape[2]) )

    i = 0
    prefix='aug__'+os.path.basename(p).replace('.png','')
    for batch in image_data_gen.flow(img_array, batch_size=1,save_to_dir=dst_dir, save_prefix=prefix, save_format='png'):
        i = i+1
        if i>=num:
            break


# 1. no-padding-resize

In [3]:
import os

dist = {}
src = 'cell_data_resize//'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '')
    if y not in dist:
        dist[y] = 0
    dist[y] += 1

In [4]:
dist

{'red_blood_cell': 2580,
 'ring': 39,
 'gametocyte': 8,
 'schizont': 11,
 'trophozoite': 5}

In [5]:
from shutil import copyfile
from tqdm import *


dst = 'cell_data_resize_aug/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell':
        copyfile(fp_src,fp_dst)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2643/2643 [00:00<00:00, 10263.79it/s]


In [6]:
# one process, too slow
from shutil import copyfile
from tqdm import *


dst = 'cell_data_resize_aug/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell' or y=='difficult':
        continue
    else:
        num_now = dist[y]
        num = dist['red_blood_cell']//num_now
        augment_one_image(fp_src, dst, num)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2643/2643 [02:27<00:00, 17.92it/s]


In [10]:
# check aug result
import os

dist = {}
src = 'cell_data_resize_aug/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '').split('_')[0]
    if y not in dist:
        dist[y] = 0
    dist[y] += 1
    
dist

{'gametocyte': 2530,
 'trophozoite': 2526,
 'ring': 2564,
 'schizont': 2541,
 'red': 2580}

In [4]:
# multiple processes
from shutil import copyfile
from tqdm import *


dst = '04_cell_data_resize_aug/'
task_para_list = []


for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell' or y=='difficult':
        continue
    else:
        num_now = dist[y]
        num = dist['red_blood_cell']//num_now
        task_para_list.append( (fp_src, dst, num) )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86035/86035 [00:00<00:00, 661092.98it/s]


In [5]:
len(task_para_list)

2555

In [12]:
import time
from multiprocessing import Process
import os


thnum = 40

def runs(sub_task_para_list):
    for xxx in sub_task_para_list:
        fp_src, dst, num = xxx
        augment_one_image(fp_src, dst, num)
        
n = len(task_para_list)//thnum
for i in range(thnum):
    p = Process(target=runs, args=(task_para_list[i*n:(i+1)*n],))  # target进程执行的任务, args传参数（元祖）
    p.start() 

In [39]:
import os
dst = '04_cell_data_resize_aug/'

category_list = ['red_blood_cell', 'ring', 'trophozoite', 'gametocyte', 'leukocyte', 'difficult', 'schizont']
count = 0
for d in os.listdir(dst):
    count += 1
print(count)

dist = {}
for d in os.listdir(dst):
    for k in category_list:
        if k in d:
            if k not in dist:
                dist[k] = 0
            dist[k] += 1
print(dist)

484516
{'red_blood_cell': 83034, 'leukocyte': 78846, 'schizont': 79470, 'gametocyte': 80317, 'ring': 82045, 'trophozoite': 80804}


In [34]:
import os
dst = '02_cell_data_cut_no_padding/'

category_list = ['red_blood_cell', 'ring', 'trophozoite', 'gametocyte', 'leukocyte', 'difficult', 'schizont']
count = 0
for d in os.listdir(dst):
    count += 1
print(count)

dist = {}
for d in os.listdir(dst):
    for k in category_list:
        if k in d:
            if k not in dist:
                dist[k] = 0
            dist[k] += 1
print(dist)

FileNotFoundError: [Errno 2] No such file or directory: '02_cell_data_cut_no_padding/'